In [1]:
from urllib.request import urlopen
url = "https://www.data.gouv.fr/fr/datasets/r/9096f970-f711-4db5-9f22-77618cb50729"
data = urlopen(url).read()

In [9]:
import json
import pandas as pd
import numpy as np
import os
import requests
import gzip
import shutil
import datetime

In [10]:
folder_gz = "data_gz"
folder_csv = "data_csv"
urls= dict()
urls["74"] = "https://www.data.gouv.fr/fr/datasets/r/9096f970-f711-4db5-9f22-77618cb50729"

In [ ]:
def convert_to_date(chaine):
    return pd.to_datetime(str(chaine), format='%Y%m%d%H', errors='coerce')

def download_file(url, filename):
    file= os.path.join(folder_gz, filename) + '.gz'
    print('Téléchargement: ', file)
    response = requests.get(url)
    if response.status_code == 200:
        with open(file, 'wb') as f:
            f.write(response.content)
    else:
        print("Fichier d'archive non présent à l'url habituelle: ", file)

def decompress_gz(filename):
    file= os.path.join(folder_gz, filename) + '.gz'
    
    if os.path.exists(file):
        with gzip.open(file, 'rb') as f_in:
            file= os.path.join(folder_csv, filename) 
            print('Décompression', file)
            with open(file, 'wb') as f_out:
                f_out.write(f_in.read())
    else:
        print("Fichier d'archive non trouvé: ", file)
        print("Téléchargez l'archive GZ, manuellement ou en modifiant la variable 'download', puis relancez le script.")

def read_csv(filename):
    file= os.path.join(folder_csv, filename)
    if os.path.exists(file):
        print('Lecture: ', file)
        df= pd.read_csv(file, header=0, sep=";", dtype={"NUM_POSTE":str, 'AAAAMMJJHH':str}, parse_dates=['AAAAMMJJHH'], date_parser= convert_to_date)
    else:
        print("Fichier CSV non trouvé: ", file)
        print("Téléchargez l'archive GZ, manuellement ou en modifiant la variable 'download', puis relancez le script")
    return df

In [8]:
# Téléchargement/décompression/lecture des fichiers dans une boucle sur les départements (urls tirées du dictionnaire 'urls')
i, j = 0, 0
departements= ["74"]
download= True
template_start= "postesEnService_"
template_end= ".csv"
for departement in departements:
    # On récupère l'url
    url = urls[departement]
    # Formation du nom du fichier à partir du template et du numéro de département
    filename = f"{template_start}{departement}{template_end}"
    if download:
        download_file(url, filename)
        decompress_gz(filename)
    # Lecture du fichier CSV dans un dataframe pandas
    df_departement= read_csv(filename)
    if i == 0: # pour le premier département, initialisation du dataframe final df       
        df= df_departement
        i= i+1
    else:  # sinon concaténation du département suivant 
        df= pd.concat([df, df_departement])    


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte